In [ ]:
import torch
from transformers import AutoModelForCausalLM
from internvl_chat import InternVL3ChatProcessor
from PIL import Image

In [ ]:
def load_model():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    processor = InternVL3ChatProcessor.from_pretrained(
        "OpenGVLab/InternVL3-14B-AWQ", 
        trust_remote_code=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        "OpenGVLab/InternVL3-14B-AWQ",
        device_map=device,
        trust_remote_code=True
    ).eval()
    
    return processor, model

In [ ]:
def main():
    processor, model = load_model()
    image_path = input().strip()
    
    try:
        img = Image.open(image_path).convert('RGB')
        device = model.device
        dtype = model.dtype
        
        pixel_values = processor(images=img, return_tensors='pt').pixel_values
        pixel_values = pixel_values.to(dtype).to(device)
        
        prompt = (
            "Describe what you see in this picture in details. "
            "Speak only English. In math tasks, 'x' is a mathematical variable, not multiplication. Example: 6x + 5 = 23. You dont need to solve them. "
            "If you see pictures with the content of weapons, drugs, violence, murders, etc., write that this picture contains prohibited content that I cannot describe"
        )
        
        inputs = processor.tokenizer(
            prompt,
            return_tensors='pt',
            add_special_tokens=True
        ).to(device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                pixel_values=pixel_values,
                max_new_tokens=1024,
                do_sample=False,
                eos_token_id=processor.tokenizer.eos_token_id
            )
        
        response = processor.tokenizer.decode(
            outputs[0][inputs.input_ids.shape[1]:], 
            skip_special_tokens=True
        )
        
        print("----------")
        print(response.strip())
        
    except Exception as e:
        print(f"Ошибка: {str(e)}")

if __name__ == "__main__":
    main()

/workspaces/.local/lib/python3.11/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/workspaces/.local/lib/python3.11/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/workspaces/.local/lib/python3.11/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
CUDA extension not installed.
CUDA extension not installed.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some weights of the model checkpoint at Qwen/Qwen-VL-Chat-Int4 were not used when initializing QWenLMHeadModel: ['transformer.h.0.attn.c_proj.bias', 'transformer.h.0.mlp.c_proj.bias', 'transformer.h.0.mlp.w1.bias', 'transformer.h.0.mlp.w2.bias', 'transformer.h.1.attn.c_proj.bias', 'transformer.h.1.mlp.c_proj.bias', 'transformer.h.1.mlp.w1.bias', 'transformer.h.1.mlp.w2.bias', 'transformer.h.10.attn.c_proj.bias', 'transformer.h.10.mlp.c_proj.bias', 'transformer.h.10.mlp.w1.bias', 'transformer.h.10.mlp.w2.bias', 'transformer.h.11.attn.c_proj.bias', 'transformer.h.11.mlp.c_proj.bias', 'transformer.h.11.mlp.w1.bias', 'transformer.h.11.mlp.w2.bias', 'transformer.h.12.attn.c_proj.bias', 'transformer.h.12.mlp.c_proj.bias', 'transformer.h.12.mlp.w1.bias', 'transformer.h.12.mlp.w2.bias', 'transformer.h.13.attn.c_proj.bias', 'transformer.h.13.mlp.c_proj.bias', 'transformer.h.13.mlp.w1.bias', 'transformer.h.13.mlp.w2.bias', 'transformer.h.14.attn.c_proj.bias', 'transformer.h.14.mlp.c_proj.bias', 

Ошибка: [Errno 2] No such file or directory: 'niki28sh2009'
